In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [6]:
print("torch version: {} \ntorchvision version: {}".format(torch.__version__, torchvision.__version__))

torch version: 1.0.1 
torchvision version: 0.2.1


In [7]:
model_names = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception', 'shufflenet']

In [ ]:
tran_filenames = os.listdir('data/train')
datasetdir = os.path.join('./data')
traindir = os.path.join(datasetdir,'train2')
testdir = os.path.join(datasetdir,'test2')

model_name = "densenet"
num_classes = 2
batch_size = 32
epochs = 10
feature_extract = True
trick = False
csv_name = "submission-{model_name}-e{epochs}b{batch_size}-t{trick}.csv"